In [86]:
import os
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from tqdm.auto import tqdm

VALIDATION_IMAGE_FOLDER = "/Users/nathan/Downloads/DRIMDB-formated"
MODEL_NAME = "model_full.h5"

In [87]:
BATCH_SIZE = 32
IMG_SIZE = (512, 512)
validation_dataset = tf.keras.utils.image_dataset_from_directory(VALIDATION_IMAGE_FOLDER, image_size=IMG_SIZE,
                                                                 batch_size=BATCH_SIZE)

Found 584 files belonging to 2 classes.


In [88]:

model = tf.keras.models.load_model(filepath=f'results/{MODEL_NAME}', compile=False)
# model = tf.keras.models.load_model(filepath="/Users/nathan/PycharmProjects/seng402-deploy/server/static/VGG16_golden_binary.h5", compile=False)

<tf.Tensor: shape=(), dtype=int64, numpy=19>

In [95]:
pred = []
labels = []
pred_AUC = []

# 3369
import time
start = time.time()
for (image_batch, label_batch) in tqdm(validation_dataset, total=validation_dataset.cardinality().numpy()):
    prediction_batch = model.predict(image_batch, verbose=0)
    for prediction in prediction_batch:
        pred_AUC.append(prediction[0])
        pred.append(prediction[0] > 0.5)
    labels.extend((label_batch.numpy() == 1).astype(int))  # Bad: 0, Good: 1, Outlier: 2
print(f"Time taken: {time.time() - start}, average {(time.time() - start)/584}")

  0%|          | 0/19 [00:00<?, ?it/s]

Time taken: 3.9388153553009033, average 0.0067445496990256116


In [90]:
print(classification_report(y_true=labels, y_pred=pred))

              precision    recall  f1-score   support

           0       0.38      1.00      0.55       208
           1       1.00      0.10      0.19       376

    accuracy                           0.42       584
   macro avg       0.69      0.55      0.37       584
weighted avg       0.78      0.42      0.32       584



In [73]:
print(confusion_matrix(y_true=labels, y_pred=pred))

[[317  59]
 [ 23 185]]


In [74]:
roc_auc_score(y_true=labels, y_score=pred)


0.8662540916530278